I am using a polytropic equation of state, which is defined as

$$ P(\varepsilon) = \Kappa\varepsilon^\gamma  $$

Since I'm using scaled units, the equation actually is written in the form

$$ P'(\varepsilon') = K\varepsilon'^{\gamma} $$

Here are the steps to transform the first equation into the second one.

These are the TOV equations.

$$
\frac{dm_r}{dr} = 4\pi r^2 \varepsilon(r)
$$

$$
\frac{dP}{dr}=
    -G\frac{m_r(r)\varepsilon(r)}{r^2}
    \left(1+\frac{P(r)}{\varepsilon(r)}\right)
    \left(1+\frac{4\pi r^3 P(r)}{m_r(r)}\right)
    \left(1-\frac{2 Gm_r(r)}{r}\right)^{-1}
$$

$r$ = radius (km).

$m_r(r)$ = the radius enclosed within a radius $r$ in ($M_\odot$).

$P(r)$ = the pressure at radius $r$ (_MeV/fm<sup>3</sup>_).

$\varepsilon(r)$ = energy density at radius $r$ (_MeV/fm<sup>3</sup>_).

$G$ = gravitational constant.



I'm going to start by using the dimensionless TOV equations.

$$
\frac{dP'}{dr'}=
    \left(-\frac{m_r' e'}{r'^2} \right)
    \left(1 + \frac{P'}{\varepsilon} \right)
    \left(1 + \frac{4\pi r'^3 P'}{m_r'} \right)
    \left(1 - \frac{2m_r'}{r'} \right)^{-1}
$$

Where a quantity being primed denotes that it is a unitless value being scaled by a constant with the units of energy density, $\varepsilon_0$. For example, $P = \varepsilon_0 P'$ (also meaning that $P' = \frac{P}{\varepsilon_0}$). To convert back to the original units, I can use the equations

$$ m_r = \frac{m_r'}{(G^3 \varepsilon_0)^\frac{1}{2}} $$

$$ r = \frac{r'}{(G\varepsilon_0)^\frac{1}{2}} $$

In [1]:
from dataclasses import dataclass
import numpy as np
from scipy.constants import pi, G
from scipy.integrate import solve_ivp

@dataclass
class MassRadiusSolutions:
    '''
    An object for encapsulating the solutions for a mass-radius curve.
    '''
    r_solutions: list[float]
    m_solutions: list[float]

    def from_solve_ivp(solutions):
        return MassRadiusSolutions(
            r_solutions=convert_radius_to_nu(solutions.t),
            m_solutions=convert_mass_to_nu(solutions.y[1])
        )

# Scaling constant
E_0 = 1

def energy_density(P_prime):
    '''
    Calculates the energy density as a function of P'.
    '''
    gamma = 2.0
    K = 1.0
    K_prime = K * (E_0 ** (gamma - 1))
    return (P_prime / K_prime)**(1/gamma)

def scaled_tov_rhs(r, state):
    '''
    Returns [dP/dr, dm/dr] using a TOV equation scaled by an energy density constant. Treat every 
    value with a unit as being dimensionless.
    '''
    # Pressure, mass, and density at the current radius
    P_r, m_r = state
    if P_r <= 0:
        return [0, 0]
    e_r = energy_density(P_r)
    # Mass 
    dm_dr = 4*pi * r**2 * e_r
    # Pressure (split into factors)
    f1 = -(m_r * e_r) / (r ** 2)
    f2 = 1 + (P_r / e_r)
    f3 = 1 + ((4*pi * (r**3) * P_r) / m_r)
    f4 = (1 - (2 * m_r / r)) ** -1
    dp_dr = f1 * f2 * f3 * f4
    return [dp_dr, dm_dr]

def pressure_boundary_event(r, state):
    '''
    Event for solve_ivp() that returns 0 when the pressure is 0,
    indicating that the edge fo the star has been reached.
    '''
    p, _ = state
    return p

# Cancel solve_ivp() when radius reaches boundary.
pressure_boundary_event.terminal = True
# Only trigger event when going from psoitive to negative
pressure_boundary_event.direction = -1

@np.vectorize
def convert_pressure_to_scaled(P):
    '''
    Returns the scaled pressure where `P` is in natural units.
    '''
    return P / E_0

@np.vectorize
def convert_mass_to_nu(m_prime):
    '''
    Returns the mass in natural units (solar masses) where `m_prime` is the scaled value.
    '''
    M_SUN = 1.98847e30
    mass_kg = m_prime / ((G**3 * E_0)**(1/2))
    return mass_kg / M_SUN

@np.vectorize
def convert_radius_to_nu(r_prime):
    '''
    Returns the radius in natural units (km) where `r_prime` is the scaled value.
    '''
    radius_meters = r_prime / ((G * E_0)**(1/2)) 
    return radius_meters / 1000

def small_r_expansion(r_0, P_c):
    return ((4*pi)/3) * r_0**3 * energy_density(P_c)

# Starting radius
R_START = 1e-6
# Maximum possible radius. solve_ivp() shoud terminate before reaching this value.
R_END = 100
# Central pressure.
P_C = 10**2
# Central enclosed mass.
M_0 = small_r_expansion(R_START, P_C) # Solar Masses

def solve() -> MassRadiusSolutions:
    '''
    Use solve_ivp() to find solutions using the scaled TOV equations. Then, the results are rescaled
    and added to a `MassRadiusSolutions` object.
    '''
    solutions = solve_ivp(
        scaled_tov_rhs,
        t_span=(R_START, R_END),
        y0=(P_C, M_0),
        events=pressure_boundary_event
    )
    return solutions

solutions = solve()
print(f"Outer Radius: {np.max(solutions.t)}")
print(f"Enclosed Mass: {np.max(solutions.y[1])}")

Outer Radius: 0.4914051572462142
Enclosed Mass: 0.14348667254157188


In [ ]:
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import matplotlib.pyplot as plt

# The graphs here aren't useful because they aren't using realistic values.

subplots: tuple[Figure, Axes] = plt.subplots()
fig, ax = subplots

ax.set_title("Mass vs. Radius")
ax.set_ylabel("Mass [M☉]")
ax.set_xlabel("Radius [km]")
ax.plot(solutions.t, solutions.y[1])

fig.tight_layout()
plt.show()